# Interatomic Potential Repository Content Manager

This is a working Notebook providing a single location for adding and modifying content in the potentials.nist.gov database.

Notebook outline:

0. Setup: Python library imports and defining global parameters.

1. Citations: Load, add and modify citation information associated with potential models. Each potential should have at least one citation, with the first being the "primary" one for generating IDs. For unpublished potentials, use the unpublished citation option.

2. Potentials: Load, add and modify potential information.  A "potential" should be a unique parameterization with an associated citation. Note that saving to the database is done in step #4.

3. Implementations: Load, add and modify the implementations of a potential. An "implementation" is a specific version/representation of a potential in a specific format. Each potential can have zero, one or multiple implementations.

4. Save potentials: Review the new/modified potentials and save the content to the database.

5. Actions: Actions are used to list changes to the repository. If a potential is added/modified, a corresponding action should also be added.

6. Requests: A request is listed on the repository whenever someone asks for a model that we do not have.

## 0. Setup

### Library imports

In [1]:
import uuid
import datetime
import shutil
from pathlib import Path

import potentials

import numpy as np

# Jupyter display libraries
from IPython.core.display import display, HTML

### Global parameters

- __complete_potentials__ is a compiled list of the potentials whose addition/changes were uploaded to the database.  This is used by step #5 for reporting website changes.  This should be reset after creating an Action for meaningful sets of potentials.

- __db__ is the Database object for accessing and uploading records to the database.  __Note__: you must have an account on potentials.nist.gov and sign in to be able to add or modify content.

- __workspace__ is the workspace to assign to the uploaded records.  If None, then they are only accessible to the submitter until the workspace is changed.

In [2]:
complete_potentials = []

In [3]:
db = potentials.Database(username='lmh1')

Enter password for lmh1 @ https://potentials.nist.gov:········
{'detail': 'Invalid username/password.'}


HTTPError: 401 Client Error: Unauthorized for url: https://potentials.nist.gov/rest/data/?title=ARBITRARYNONEXISTANTTITLE

In [4]:
#workspace = None
workspace = db.cdcs.global_workspace

- - -

## 1. Citations

In [5]:
db.load_citations(verbose=True)

Loaded 333 local citations
Loaded 333 remote citations
 - 0 new


### 1.1 Build or load citation

#### Option #1: For existing citations and/or new dois

In [6]:
doi = '10.5555/12345678'

citation = db.get_citation(doi, verbose=True)

Citation retrieved from CrossRef


#### Option #2: For new doi-less citations

In [5]:
author = 'Bob Test'
year = 2099
ID = note = f'{year}--Test-L-M'

citation = potentials.Citation(ENTRYTYPE = 'unpublished',
                               title = 'to be published',
                               author = author,
                               ID = ID,
                               note = note,
                               year = year)

#### Option #3: From local directory

In [11]:
libdir = Path('E:/website/LAMMPS potentials/2nn/biblib')

citations = []
for bibfile in libdir.glob('*.bib'):
    with open(bibfile) as f:
        citations.append(potentials.Citation(model=f.read()))

In [ ]:
citation = citations[19]
print(citation.year_authors)

### 1.2 Investigate and modify

- Add abstract

- Check and replace garbled latex or other symbols in authors, title and abstract

- Verify correct year and page number(s)

#### 1.2.1 Generate html and list content fields

In [7]:
print(citation.year_authors)
print()
display(HTML(citation.html()))
print()
for key, value in citation.asdict().items():
    print(key+':', value)

2008--Carberry-J




ENTRYTYPE: article
ID: Carberry_2008
author: Josiah Carberry
doi: 10.5555/12345678
journal: Journal of Psychoceramics
month: aug
number: 11
pages: 1--3
publisher: Society of Psychoceramics
title: Toward a Unified Theory of High-Energy Metaphysics: Silly String Theory
url: https://doi.org/10.5555%2F12345678
volume: 5
year: 2008


#### 1.2.2 Modify and update fields

Any valid bibtex fields can be added, but citation rendering relies only on the primary fields. 

In [8]:
citation.abstract = ' '.join(['Testing afs'])
#citation.author = 'I Sa and B Lee'
#citation.volume
#citation.number
#citation.title = 'Applications of computational thermodynamics - the extension from phase equilibrium to phase transformations and other properties'
#citation.year
#citation.pages

#### 1.2.3 Review and Save Citation

In [9]:
display(HTML(citation.html()))

In [10]:
db.save_citations(citation, overwrite=False, verbose=True)

1 citations saved to localpath
 - 1 new citations added


In [11]:
db.upload_citation(citation, workspace=workspace, verbose=True)

record 10.5555_12345678 (5f4d10295b91e500396cd52b) successfully uploaded.
record 5f4d10295b91e500396cd52b assigned to workspace 5de9613a75d7d40014ffb6fc


In [12]:
#db.delete_citation(citation, local=True, remote=True, verbose=True)

deleted C:\Users\lmh1\Documents\library\Citation\10.5555_12345678.bib
record 10.5555_12345678 (5f4d10295b91e500396cd52b) has been deleted.


In [15]:
db.download_citations(overwrite=True, verbose=True)

333 citations saved to localpath
 - 333 duplicate citations skipped


- - -

## 2. Potentials

In [10]:
db.load_potentials(verbose=True)

Loaded 294 local potentials
Loaded 294 remote potentials
 - 0 new


### 2.1 Build or load potential

#### Option #1: Build new potential using citation(s)

- __elements__ (*list*) The element models used by the potential.
- __citations__ (*list*) The citation(s) to associate with the potential.
- __notes__ (*str, optional*) Notes providing more information on the potential, such as usage notes and how the potential relates to other potentials.
- __key__ (*str, optional*) UUID4 key to assign to the potential.  If None, a new key will be generated.
- __othername__ (*str, optional*) Specifies an alternate description of the interactions besides the list of elements.  Examples include if the model is "universal", designed for a specific compound, or is a coarse-grain representation.
- __fictional__ (*bool, optional*) Flag indicating if the model is "fictional", i.e. purposefully fit to unrealistic properties. 
- __modelname__ (*str, optional*) Used if citation + element info is not enough to create a unique ID, e.g. a publication lists multiple parameterizations of the same element. The value should make it clear which version it is associated with, and correspond to how the authors refer to it when possible.
- __recorddate__ (*datetime.date, optional*) Date to assign to the record.  If not given, will use today's date.

In [11]:
potential = potentials.Potential(
    elements=['Au', 'Rh'],
    citations=[citation],
    #notes=None,
    #key=None,
    #othername='water',
    #fictional=True,
    #modelname='34',
    #recorddate=datetime.date(2019, 12, 24)
    )

#### Option #2: Select existing potential

In [13]:
potential = db.get_potential(id='2008--Carberry-J--Au-Rh')

### 2.2 Investigate and modify

#### 2.2.1 Generate html and list content fields

In [12]:
display(HTML(potential.html()))
print()
for key, value in potential.asdict().items():
    print(key+':', value)


key: 242c630d-017d-4212-a5c3-9c9f81bc1c29
id: 2008--Carberry-J--Au-Rh
recorddate: 2020-08-31
notes: None
fictional: False
elements: ['Au', 'Rh']
othername: None
modelname: None
citations: [<potentials.Citation.Citation object at 0x0000022DF7592F28>]
implementations: []


#### 2.2.2 Modify and update fields

Same fields as above

In [22]:
#potential.key = 
#potential.recorddate = datetime.date()
#potential.elements = []
#potential.othername = 'water'
#potential.fictional = 
#potential.modelname = 

In [8]:
potential.notes = None

In [13]:
potential.notes = ' '.join(["This listing is for the MCu34 parameterization listed in the reference.",
                            "Dr. M.I. Mendelev (Ames Laboratory) noted that these are new fictional potentials intended",
                            "to supplement the existing potentials posted to the NIST repository",
                            "(as the 2015--Borovikov-V-Mendelev-M-I-King-A-H-LeSar-R--fictional-Cu-# listings).",
                            'Dr. Mendelev further noted that, "the new potentials provide the same SFE as',
                            '<a href="https://www.ctcms.nist.gov/potentials/entry/2013--Mendelev-M-I-King-A-H--Cu/">2013--Mendelev-M-I-King-A-H--Cu</a>',
                            'but different unstable stacking fault',
                            'energy (USFE). All these Cu fictional potentials are designed to study the effect of SFE and USFE on the',
                            'deformation behavior in fcc metals."  Reference information added March 5, 2020.'])         

In [ ]:
#potential.notes += 

## 3. Implementations

### 3.1 List all current implementation types

List is given to help keep type values (relatively) uniform. 

In [14]:
imptypes = []
for pot in db.potentials:
    for imp in pot.implementations:
        imptypes.append(imp.type)
imptypes = np.unique(imptypes)

print('All existing implementation types:')
for imptypes in imptypes:
    print('   ', imptypes)

All existing implementation types:
    ADP tabulated functions
    Dynamo MEAM
    EAM setfl
    EAM tabulated functions
    Equations
    FORTRAN
    Finnis-Sinclair tables
    GULP
    IMD option EAM
    LAMMPS pair_style adp
    LAMMPS pair_style aenet (custom)
    LAMMPS pair_style agni
    LAMMPS pair_style bop
    LAMMPS pair_style comb3
    LAMMPS pair_style eam
    LAMMPS pair_style eam/alloy
    LAMMPS pair_style eam/cd
    LAMMPS pair_style eam/fs
    LAMMPS pair_style edip
    LAMMPS pair_style edip/multi
    LAMMPS pair_style eim
    LAMMPS pair_style extep
    LAMMPS pair_style hybrid table linear 1000 eam/alloy
    LAMMPS pair_style hybrid/overlay eam/alloy eam/fs
    LAMMPS pair_style hybrid/overlay zbl eam/alloy
    LAMMPS pair_style hybrid/overlay zbl snap
    LAMMPS pair_style lcbop
    LAMMPS pair_style meam
    LAMMPS pair_style meam (modified)
    LAMMPS pair_style meam/spline
    LAMMPS pair_style polymorphic
    LAMMPS pair_style reax/c
    LAMMPS pair_style sw
 

### 3.2 Build or load implementation

#### 3.2.1 List ids for current potential and implementations

In [15]:
print('potential id:', potential.id)
print('implementation ids:')
for i, implementation in enumerate(potential.implementations):
    print(f'{i}: {implementation.id}')

potential id: 2008--Carberry-J--Au-Rh
implementation ids:


#### 3.2.2 Build new implementation

- __type__ (*str*) Describes the type of the implementation. Use one of the listed values above if possible.
- __id__ (*str*) Unique human-readable id for the implementation. For new content, should be derived from the associated potential id:
    - Remove all authors except for the first
    - Add a simple format descriptor: LAMMPS, GULP, table, parameters, FORTRAN, etc.
    - Add a version descriptor: ipr1, ipr2, etc
- __notes__ (*str, optional*) Notes on the implementation.  This includes where the file(s) came from, who created them, testing info, how it differs from other versions, etc.
- __key__ (*str, optional*) UUID4 key to assign to the implementation.  If None, a new key will be generated.
- __status__ (*str, optional*) Indicates the status of the implementation. Available values are 
    - "active" (default) indicates a current implementation.
    - "superseded" indicates an implementation that is still consistent with the potential model, but has minor issues that were fixed by a newer implementation.
    - "retracted" indicates an implementation that was identified as being an invalid representation of the potential model.
- __date__ (*datetime.date, optional*) The date that the implementation was submitted or added.  If not given, will use today's date.

In [16]:
implementation = potentials.Implementation(
    type='LAMMPS pair_style adp',
    id='2008--Carberry-J--Au-Rh--LAMMPS--ipr1',
    notes=' '.join(['This file was provided by Gang Wang (Universität Stuttgart) on August 18, 2020 and posted with his permission.']),
    #key='0cdc699f-e78a-49db-8606-7a33c98c2184',
    #status=None,
    #date=datetime.date(2020, 6, 18), 
)
index = len(potential.implementations)
potential.implementations.append(implementation)

#### 3.2.3 Select implementation

Must be done

In [17]:
index = 0
implementation = potential.implementations[index]

### 3.3 Modify implementation

#### 3.3.1 Modify implementation metadata 

In [18]:
display(HTML(implementation.html()))

In [18]:
implementation.notes = 'This file was provided by Gang Wang (University of Science and Technology Beijing) on August 18, 2020 and posted with his permission.'

In [33]:
implementation.status='superseded'

In [17]:
#implementation.type='LAMMPS pair_style sw',
#implementation.key=None,
#implementation.id='2009--Molinero-V--water--ipr-1',
#implementation.status=None,
#implementation.date=datetime.date(2020, 1, 10), 
implementation.notes= ' '.join(['The table files were sent by Xianbao Duan (Huazhong Univ. of Sci. and Tech) on 18 June 2020 and posted with his permission. The example.lammps.in file gives an example of the LAMMPS pair_style and pair_coeff lines that can be used. A copy of Ta_Zhou04.eam.alloy from 2004--Zhou-X-W--Ta--LAMMPS--ipr2 is included here for completeness.',
                                ])

In [191]:
implementation.date=datetime.date(2014, 11, 24)

In [188]:
implementation.notes = 'This file was provided by Sergei Starikov (Ruhr-Universität Bochum, Germany) on 5 May 2019. It has been carefully tested and gives the expected property predictions. Update March 15, 2020: This version was identified to not be compatible with LAMMPS versions after 7 Aug 2019 due to more rigorous format checks.'

In [135]:
implementation.notes += ' Update Jan 15, 2020: It was noticed that the original file hosted here was truncated and incomplete.'
implementation.notes += ' The incomplete file will not work with LAMMPS versions after 7 Aug 2019.'
implementation.notes += ' For earlier LAMMPS versions, both versions of the parameter file appear to behave identically.'

In [50]:
implementation.status = 'retracted'
implementation.notes = 'This file was sent by J. Kullgren (Uppsala University) on 19 December 2016 and posted with his permission. Update March 15, 2020: This version was identified to not be compatible with LAMMPS.'

In [53]:
implementation.notes='These files were posted on March 15, 2020 by Lucas Hale. They modify the above version by separating the comments into a separate file, making the parameter file compatible with LAMMPS.'

In [162]:
implementation.type = 'LAMMPS pair_style reax/c'

In [203]:
implementation.id = '2013--Bonny-G--Fe-Cr-W--LAMMPS--ipr1'

#### 3.3.2 Add implementation content

The content for the implementation can exist in one of the following formats

1. Artifacts: files that are hosted on the potentials repository.
2. Parameters: list of the potential's parameters to show as html text on the repository.
3. WebLinks: hyperlinks to content hosted externally.

#### Option #1: Add artifacts

Each artifact has:

- __filename__ (*str*) The name of the file (no path information).
- __url__ (*str*) The absolute url where the file can be downloaded from.
- __label__ (*str, optional*) An optional descriptive label for the file.

In [19]:
# Define paths to files
machine = 'desktop'

if machine == 'desktop':
    newpotpath = Path('E:/website/new potentials')
    localpath = Path('E:/website/IPR-website/potentials/')
elif machine == 'laptop':
    newpotpath = Path('C:/Users/lmh1/Documents/website/new potentials')
    localpath = Path('C:/Users/lmh1/Documents/website/IPR-website/potentials/')
    
webpath = 'https://www.ctcms.nist.gov/potentials/'
relpath = f'Download/{potential.id}/{index+1}/'

In [20]:
# Specify files from newpotpath to add
filenames = [
    Path(newpotpath, 'AuRh.adp.txt'),
    #Path(newpotpath, 'comments.txt'),
    ]

labels = [None for i in range(len(filenames))]

In [28]:
# Specify files from newpotpath to add
filenames = [
    Path(newpotpath, 'He.Aziz.table'),
    Path(newpotpath, 'TaHe.Juslin.table'),
    Path(newpotpath, 'Ta_Zhou04.eam.alloy'),
    Path(newpotpath, 'example.lammps.in'),
    ]

labels = [None for i in range(len(filenames))]

In [30]:
# Make download directory in local copy of website
downloadpath = Path(localpath, relpath)
if not downloadpath.is_dir():
    downloadpath.mkdir(parents=True)

# Copy files and add artifact listings
for filename, label in zip(filenames, labels):
    # Build url
    url = webpath + relpath + filename.name
    # Copy files
    shutil.copy(filename, downloadpath)
    print(Path(downloadpath, filename.name))
    
    # Add artifact listing to implementation
    implementation.add_artifact(filename=filename.name, url=url, label=label)

E:\website\IPR-website\potentials\Download\2021--Wang-G-Xu-Y-Qian-P-Su-Y--Au-Rh\1\AuRh.adp.txt


In [20]:
implementation.artifacts[0].url

'https://www.ctcms.nist.gov/potentials/Download/1987--Ackland-G-J-Thetford-R--Mo/4/mo.fs.eam.alloy'

In [151]:
implementation.artifacts[0].url = 'https://www.ctcms.nist.gov/potentials/Download/2016--Wilson-S-R-Mendelev-M-I--fictional-W/1/W2.eam.fs'

#### Option #2: Add parameters

Each parameter has:

- __name__ (*str, optional*) The name of the parameter or string parameter line.
- __value__ (*float, optional*) The value of the parameter.
- __unit__ (*str, optional*) Units associated with value.

Note: if the parameters are purely text-based, you can use only the name field.

#### Option #3: Add weblinks

Each weblink has:

- __url__ (*str*) URL for the link.
- __label__ (*str, optional*) A short description label to proceed the link.
- __linktext__ (*str, optional*) The text for the link, i.e. what gets clicked on.

## 4. Review and save potentials

In [21]:
display(HTML(potential.html()))

In [20]:
db.upload_potential(potential, workspace=workspace, verbose=True)
complete_potentials.append(potential)

record potential.2008--Carberry-J--Au-Rh (5f4d132b5b91e500396cd52f) successfully uploaded.
record 5f4d132b5b91e500396cd52f assigned to workspace 5de9613a75d7d40014ffb6fc


In [21]:
db.save_potentials(potential, format='json', indent=4, verbose=True)

1 potentials saved to localpath
 - 1 new potentials added


In [22]:
#db.delete_potential(potential, local=True, remote=True, verbose=True)

deleted C:\Users\lmh1\Documents\library\Potential\potential.2008--Carberry-J--Au-Rh.json
record potential.2008--Carberry-J--Au-Rh (5f4d132b5b91e500396cd52f) has been deleted.


In [23]:
db.download_potentials(overwrite=True, format='json', indent=4, verbose=True)

294 potentials saved to localpath
 - 294 duplicate potentials skipped


## 5. potential_LAMMPS

This section generates the potential_LAMMPS metadata records for generating the LAMMPS input command lines for supported potentials.

### 5.1 Build potential_LAMMPS

#### Specify parameters

Common settings

- __id__ (*str, optional*) implementation id. Should match with the implementation id from above.
- __key__ (*str, optional*) implementation key. Should match with the implementation key from above.
- __potid__ (*str, optional*) potential id. Should match with the potential id from above.
- __potkey__ (*str, optional*) potential key. Should match with the potential key from above.
- __elements__ (*list, optional*) The elements that the implementation simulates.  Should match with the elements from above if the implementation explicitly models each element.
- __symbols__ (*list, optional*) The particle model symbols defined by the implementation.  Not needed if identical to elements.
- __masses__ (*list, optional*) The masses to assign to each symbol/element.  Should be given if the implementation specifies the mass.  Required if elements is not given.
- __units__ (*str, optional*) The LAMMPS units to use with the implementation.  Default value is 'metal'.
- __atom_style__ (*str, optional*) The LAMMPS atom_style to use with the implementation.  Default value is 'atomic'.
- __pair_style__ (*str*) The LAMMPS pair_style associated with the implementation.
- __pair_style_terms__ (*list, optional*) Any additional terms that should appear on the pair_style command line.

In [22]:
kwargs = {}
kwargs['id'] = implementation.id
kwargs['key'] = implementation.key
kwargs['potid'] = potential.id
kwargs['potkey'] = potential.key

kwargs['elements'] = potential.elements
#kwargs['symbols'] = 
kwargs['masses'] = [196.9666, 102.9055]

#kwargs['units'] =
#kwargs['atom_style'] = 

kwargs['pair_style'] = 'adp'
#kwargs['pair_style_terms'] = ['polar_off']

Option #1 For pair potentials without parameter files, e.g lj/cut

- __interactions__ (*list*) Lists parameters associated with each unique interaction.  Each value is a dict with
    - __symbols__ (*list*) The two elemental model symbols to associate with the interaction.
    - __terms__ (*list*) The terms that appear on the pair_coeff command line for that interaction.

In [ ]:
kwargs['interactions'] = [
    {'symbols': ['Al', 'Al'], 'terms': [1.23, 3.412]},
    {'symbols': ['Al', 'Cu'], 'terms': [1.23, 3.412]},
    {'symbols': ['Cu', 'Cu'], 'terms': [1.23, 3.412]},
]

Option #2 For potentials with single parameter files, e.g. eam/alloy

- __paramfile__ (*str*) The name (no path) of the potential's parameter file.

In [23]:
for file in filenames:
    print(file.name)

AuRh.adp.txt


In [24]:
kwargs['paramfile'] = 'AuRh.adp.txt'

Option #3 For potentials with a parameter file and a library file, e.g. meam

- __libfile__ (*str*) The name (no path) of the potential's library parameter file.
- __paramfile__ (*str, optional*) The name (no path) of the potential's parameter file, if one is used.

In [24]:
kwargs['libfile'] = 'library.meam'
kwargs['paramfile'] = 'parameter.meam'

Option #4 For classic EAM potentials, i.e. eam

- __paramfiles__ (*list*) The names (no paths) of the parameter files for each unique element. Order should match order of elements.

In [ ]:
kwargs['paramfiles'] = []

Option #5 For OpenKIM potentials

- __kimid__ (*str*) The KIM model ID.

In [ ]:
kwargs['kimid'] = 

#### Build potential_LAMMPS

In [25]:
lammps_potential = potentials.build_lammps_potential(**kwargs).potential()

### 5.2 Load potential_LAMMPS from database 

In [5]:
lammps_potential = db.get_lammps_potential(id='2015--Broqvist-P--Ce-O--LAMMPS--ipr2')

### 5.3 Load potential_LAMMPS from local file not in database 

In [27]:
fname = Path('C:/Users/lmh1/Documents/library/potential_LAMMPS/2019--Fischer-F--Cu-Ni--LAMMPS--ipr2.json')

lammps_potential = potentials.PotentialLAMMPS(fname)

### 5.4 Review and save

#### Review model

In [26]:
print(lammps_potential.asmodel().json(indent=2))

{
  "potential-LAMMPS": {
    "key": "607ab2d7-7f32-4556-9f12-d5c3304e7eab", 
    "id": "2008--Carberry-J--Au-Rh--LAMMPS--ipr1", 
    "potential": {
      "key": "242c630d-017d-4212-a5c3-9c9f81bc1c29", 
      "id": "2008--Carberry-J--Au-Rh"
    }, 
    "units": "metal", 
    "atom_style": "atomic", 
    "atom": [
      {
        "element": "Au", 
        "mass": 196.9666
      }, 
      {
        "element": "Rh", 
        "mass": 102.9055
      }
    ], 
    "pair_style": {
      "type": "adp"
    }, 
    "pair_coeff": {
      "term": [
        {
          "file": "AuRh.adp.txt"
        }, 
        {
          "symbols": "True"
        }
      ]
    }
  }
}


#### Save locally

In [27]:
db.save_lammps_potentials(lammps_potential, filenames=filenames, format='json',
                          indent=4, overwrite=True, verbose=True)

1 LAMMPS potentials saved to localpath
 - 1 new potentials added
 - 1 potentials had files copied


#### Upload to potentials.nist.gov

In [28]:
db.upload_lammps_potential(lammps_potential, workspace=workspace, verbose=True)

record 2008--Carberry-J--Au-Rh--LAMMPS--ipr1 (5f4d14e25b91e500386cd528) successfully uploaded.
record 5f4d14e25b91e500386cd528 assigned to workspace 5de9613a75d7d40014ffb6fc


In [29]:
db.delete_lammps_potential(lammps_potential, local=True, remote=True, verbose=True)

deleted C:\Users\lmh1\Documents\library\potential_LAMMPS\2008--Carberry-J--Au-Rh--LAMMPS--ipr1.json
record 2008--Carberry-J--Au-Rh--LAMMPS--ipr1 (5f4d14e25b91e500386cd528) has been deleted.


In [30]:
db.download_lammps_potentials(format='json', indent=4, getfiles=True, overwrite=True, verbose=True)

279 LAMMPS potentials saved to localpath
 - 1 existing potentials updated
 - 278 duplicate potentials skipped
Files for 1 LAMMPS potentials downloaded


- - -

## 5. Actions

Actions are used by the website to note changes in content and representation. 

### 5.1 Build action

#### Option #1: new posting(s)

In [22]:
for complete_potential in complete_potentials:
    print(complete_potential.id)

2021--Wang-G-Xu-Y-Qian-P-Su-Y--Au-Rh


In [23]:
action = potentials.Action(
    type = 'new posting',
    potentials = complete_potentials,
   # date = None,
    comment = 'New posting for Au-Rh',
)

#### Option #2: updated posting(s)

In [26]:
action = potentials.Action(
    type = 'updated posting',
    potentials = complete_potentials,
    date = None,
    comment = 'LAMMPS-compatible version added',
)

#### Option #3: retraction

In [ ]:
action = potentials.Action(
    type = 'retraction',
    potentials = complete_potentials,
    date = None,        
    comment =,
)

#### Option #4: site change

In [193]:
action = potentials.Action(
    type = 'site change',
    date = None,
    comment = 'IDs added for all non-LAMMPS implementations. The handling of fictional potentials made consistent with "real" potentials.',
    potentials = []
)

### 5.2 Look and save

In [24]:
display(HTML(action.html()))

In [25]:
title = f"{action.date} {action.comment[:90]}"
content = action.asmodel().xml()
template = 'Action'
db.cdcs.upload_record(content=content, template=template, title=title, workspace=workspace)

record 2020-08-25 New posting for Au-Rh (5f459a29ab2f7c0033c3a350) successfully uploaded.


- - -

## 6. Requests

Requests are used by the website to list when people are looking for specific models that we don't have.

### 6.1 Build request

In [63]:
request = potentials.Request(
    #comment = 'reactive potentials',
    date = datetime.date(2020, 1, 2),
    systems = [
        {
            #'formula':,
            'elements': ['Mg', 'Cu'],
        },
        #{
        #    'formula': 'Moose',
       #     'elements': ['Ti', 'Cl'],
       # },
    ]
)

### 6.2 Look and save

In [64]:
display(HTML(request.html()))

In [65]:
model = request.asmodel()
elements = ' '.join(model.finds('element'))

title = f'{request.date} {elements}'
content = model.xml()
template = 'Request'

db.cdcs.upload_record(content=content, template=template, title=title, workspace=workspace)

record 2020-01-02 Mg Cu (5e5eb358ab2f7c0033011111) successfully uploaded.


## 7. Related models

In [61]:
import json

def add_related_models(localpath, interaction, model1, model2):
    
    with open(Path(localpath, 'site/related-interactions.json')) as f:
        related_models = json.load(f)
    
    # If no shared models for that interaction yet
    if interaction not in related_models:
        print('new interaction')
        related_models[interaction] = [[model1, model2]]
            
    else:
        int_models = related_models[interaction]
        
        # Search for set with one of the elements
        match = False
        for i in range(len(int_models)):
            modelset = int_models[i]
            if model1 in modelset:
                if match is False:
                    match = i
                else:
                    raise ValueError('given models found in multiple sets!')
            if model2 in modelset:
                if match is False or match == i:
                    match = i
                else:
                    raise ValueError('given models found in multiple sets!')
                    
        if match is False:
            print('new set')
            int_models.append(sorted([model1, model2]))
        else:
            if model1 not in int_models[match]:
                int_models[match] = sorted(int_models[match] + [model1])
                print('model1 added to existing set')
            elif model2 not in int_models[match]:
                int_models[match] = sorted(int_models[match] + [model2])
                print('model2 added to existing set')
            else:
                print('both models already in existing set')
    
    with open(Path(localpath, 'site/related-interactions.json'), 'w') as f:
        json.dump(related_models, fp=f)

In [63]:
interaction = 'Ni'
model1 = '2004--Mishin-Y--Ni-Al'
model2 = '2019--Fischer-F-Schmitz-G-Eich-S-M--Cu-Ni'
machine = 'desktop'

if machine == 'desktop':
    localpath = Path('E:/website/IPR-website/potentials/')
elif machine == 'laptop':
    localpath = Path('C:/Users/lmh1/Documents/website/IPR-website/potentials/')

add_related_models(localpath, interaction, model1, model2)

model2 added to existing set
